booster = gbtree, target clipping, early_stopping_rounds = 10
https://xgboost.readthedocs.io/en/latest/python/python_intro.html#training

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

import time

In [3]:
data = pd.read_pickle('./data/data_1.pkl')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8032887 entries, 827691 to 8860577
Data columns (total 29 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   month_id                    int8   
 1   shop_id                     int8   
 2   item_id                     int16  
 3   cnt_month                   float32
 4   city_id                     int8   
 5   cat_id                      int8   
 6   main_cat                    int8   
 7   sub_cat                     int8   
 8   year                        int16  
 9   month                       int8   
 10  sales_month                 int8   
 11  cnt_month_1                 float32
 12  cnt_month_2                 float32
 13  cnt_month_3                 float32
 14  item_id_mean_cnt_1          float32
 15  item_id_mean_cnt_2          float32
 16  item_id_mean_cnt_3          float32
 17  sub_cat_mean_cnt_1          float32
 18  sub_cat_mean_cnt_2          float32
 19  sub_cat_mean_cnt

In [4]:
DATA = data.copy()

In [5]:
data.columns

Index(['month_id', 'shop_id', 'item_id', 'cnt_month', 'city_id', 'cat_id',
       'main_cat', 'sub_cat', 'year', 'month', 'sales_month', 'cnt_month_1',
       'cnt_month_2', 'cnt_month_3', 'item_id_mean_cnt_1',
       'item_id_mean_cnt_2', 'item_id_mean_cnt_3', 'sub_cat_mean_cnt_1',
       'sub_cat_mean_cnt_2', 'sub_cat_mean_cnt_3', 'price_mean_mean_cnt_1',
       'price_mean_mean_cnt_2', 'price_mean_mean_cnt_3',
       'item_id/city_id_mean_cnt_1', 'item_id/city_id_mean_cnt_2',
       'item_id/city_id_mean_cnt_3', 'shop_id/cat_id_mean_cnt_1',
       'shop_id/cat_id_mean_cnt_2', 'shop_id/cat_id_mean_cnt_3'],
      dtype='object')

feature only _1

In [7]:
#사용 feature 설정

features = ['month_id', 'shop_id', 'item_id', 'city_id', 'cat_id','main_cat', 'sub_cat', 'year', 'month', 'sales_month', 'cnt_month_1', 'item_id_mean_cnt_1', 'sub_cat_mean_cnt_1','price_mean_mean_cnt_1','item_id/city_id_mean_cnt_1', 'item_id/city_id_mean_cnt_2','shop_id/cat_id_mean_cnt_1', 'cnt_month']

data= data[features]


clip only target. no validation

In [8]:
# train/validation/test data split, target clip
X_train = data[data['month_id'] < 34].drop(['cnt_month'], axis=1)
y_train = data[data['month_id'] < 34]['cnt_month'].clip(0,20)

X_test = data[data['month_id'] == 34].drop(['cnt_month'], axis=1)

In [9]:
X_train.shape, y_train.shape,X_test.shape

((7818687, 17), (7818687,), (214200, 17))

In [11]:
xgb_model = XGBRegressor()

In [12]:
start = time.time()
xgb_model.fit(X_train, 
              y_train, 
              eval_metric="rmse", 
              eval_set=[(X_train, y_train)], 
              verbose=10, early_stopping_rounds = 10)  #n_estimate 수만큼 반복하지 않더라도 일정 횟수에서 더이상 cost function 값(예측 오류)가 감소하지 않으면 중단하는 최대 반복 횟수

[0]	validation_0-rmse:1.09771
[10]	validation_0-rmse:0.89209
[20]	validation_0-rmse:0.87369
[30]	validation_0-rmse:0.86315
[40]	validation_0-rmse:0.85543
[50]	validation_0-rmse:0.84692
[60]	validation_0-rmse:0.84161
[70]	validation_0-rmse:0.83789
[80]	validation_0-rmse:0.83198
[90]	validation_0-rmse:0.82928
[99]	validation_0-rmse:0.82549


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=4,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='approx', validate_parameters=1, verbosity=None)

In [13]:
#time
end = time.time()
print(f'소요시간(s) : {end-start}')

소요시간(s) : 878.0471119880676


In [14]:
#prediction, validation
train_pred = xgb_model.predict(X_train).clip(0, 20)
test_pred = xgb_model.predict(X_test).clip(0, 20)

print('Train rmse:', np.sqrt(mean_squared_error(y_train, train_pred)))


Train rmse: 0.8264385


In [16]:
# 제출 파일 생성
df_test = pd.read_csv('./data/test.csv')
submission = pd.DataFrame({"ID": df_test.index, "item_cnt_month": test_pred})
submission.to_csv('sek_xgb1_3_submit.csv', index=False)